In [36]:
# ResNet
# residual block = path1(Conv2d + ReLU + Conv2d) + path2(identity) -> activation(ReLU)

from torchsummary import summary

import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1)
        )
        self.skip_path = nn.Identity()
        self.out_act = nn.ReLU()
    def forward(self, x):
        x_conv = self.conv_path(x)
        x_skip = self.skip_path(x)
        x = x_conv + x_skip # concat이 아니라 값을 더한다.
        x = self.out_act(x)
        return x

block = ResidualBlock(in_channels=64, out_channels=64)
summary(block, (64, 56, 56))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]          36,928
              ReLU-2           [-1, 64, 56, 56]               0
            Conv2d-3           [-1, 64, 56, 56]          36,928
          Identity-4           [-1, 64, 56, 56]               0
              ReLU-5           [-1, 64, 56, 56]               0
Total params: 73,856
Trainable params: 73,856
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.77
Forward/backward pass size (MB): 7.66
Params size (MB): 0.28
Estimated Total Size (MB): 8.70
----------------------------------------------------------------


In [124]:
from torchsummary import summary
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1)
        )
        self.skip_path = nn.Identity()
        self.out_act = nn.ReLU()
    def forward(self, x):
        x_conv = self.conv_path(x)
        x_skip = self.skip_path(x)
        x = x_conv + x_skip
        x = self.out_act(x)
        return x

class ResidualBlockDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlockDown, self).__init__()
        self.conv_path = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1, stride=1)
        )
        self.skip_path = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, padding=0, stride=2)
        self.out_act = nn.ReLU()
    def forward(self, x):
        x_conv = self.conv_path(x)
        x_skip = self.skip_path(x)
        x = x_conv + x_skip
        x = self.out_act(x)
        return x

class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, padding=3, stride=2),
            nn.ReLU()
        )
        self.conv2 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, padding=1, stride=2),
            ResidualBlock(in_channels=64, out_channels=64),
            ResidualBlock(in_channels=64, out_channels=64),
            ResidualBlock(in_channels=64, out_channels=64)
            # ResidualBlockDown(in_channels=64, out_channels=128)
        )
        self.conv3 = nn.Sequential(
            ResidualBlockDown(in_channels=64, out_channels=128),
            ResidualBlock(in_channels=128, out_channels=128),
            ResidualBlock(in_channels=128, out_channels=128),
            ResidualBlock(in_channels=128, out_channels=128)
            # ResidualBlockDown(in_channels=128, out_channels=256)
        )
        self.conv4 = nn.Sequential(
            # ResidualBlockDown(in_channels=128, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlock(in_channels=256, out_channels=256),
            ResidualBlockDown(in_channels=256, out_channels=512)
        )
        self.conv5 = nn.Sequential(
            # ResidualBlockDown(in_channels=256, out_channels=512),
            ResidualBlock(in_channels=512, out_channels=512),
            ResidualBlock(in_channels=512, out_channels=512),
            ResidualBlockDown(in_channels=512, out_channels=1024)
        )
        self.avg_pool = nn.AvgPool2d(kernel_size=7, padding=0, stride=1)
        self.fc = nn.Linear(in_features=1024, out_features=1000)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        print(x.shape)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avg_pool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

model = ResNet34()
summary(model, (3, 224, 224))

torch.Size([2, 64, 56, 56])


RuntimeError: Given groups=1, weight of size [256, 256, 3, 3], expected input[2, 128, 28, 28] to have 256 channels, but got 128 channels instead